In [84]:
import torch
from torch import nn, optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

In [85]:
def one_hot(c: int, vocab_size: int):
    encoded = torch.zeros(vocab_size)
    encoded[c] = 1.0
    return encoded

In [86]:
class SequenceDataset(Dataset):
    def __init__(self, txt_path: str, seq_length: int = 100) -> None:
        f = open(txt_path, encoding="utf-8")
        self.seq_length = seq_length
        self.corpus = "".join(f.readlines()).lower()
        self.vocab = set(self.corpus)
        self.vocab_to_idx = {i: idx for (idx, i) in enumerate(self.vocab)}
        self.idx_to_vocab = {v: k for k, v in self.vocab_to_idx.items()}
        self.corpus_encoded = torch.zeros(len(self.corpus), len(self.vocab))

        for i, char in tqdm(enumerate(self.corpus), total=len(self.corpus), position=0, leave=True):
            self.corpus_encoded[i] = one_hot(self.vocab_to_idx[char], len(self.vocab))

        self.xvals = self.corpus_encoded[:-1]
        self.yvals = self.corpus_encoded[1:]

        extra_vals = len(self.xvals) % seq_length

        self.xvals = self.xvals[:-extra_vals].view(len(self.xvals) // seq_length, seq_length, len(self.vocab))
        self.yvals = self.yvals[:-extra_vals].view(len(self.yvals) // seq_length, seq_length, len(self.vocab))


    def __len__(self) -> int:
        return self.xvals.shape[0]

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.xvals[idx], self.yvals[idx]

In [87]:
class LSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim):
        super(LSTMCell, self).__init__()
        self.input_dim, self.hidden_dim = input_dim, hidden_dim
        self.forget_input, self.forget_hidden, self.forget_bias = self.create_gate_parameters()
        self.input_input, self.input_hidden, self.input_bias = self.create_gate_parameters()
        self.output_input, self.output_hidden, self.output_bias = self.create_gate_parameters()
        self.cell_input, self.cell_hidden, self.cell_bias = self.create_gate_parameters()

    def create_gate_parameters(self):
        input_weights = nn.Parameter(torch.zeros(self.input_dim, self.hidden_dim))
        hidden_weights = nn.Parameter(torch.zeros(self.hidden_dim, self.hidden_dim))
        nn.init.xavier_uniform_(input_weights)
        nn.init.xavier_uniform_(hidden_weights)
        bias = nn.Parameter(torch.zeros(self.hidden_dim))
        return input_weights, hidden_weights, bias

    def forward(self, x, h, c):
        # x has shape [batch_size, seq_len, input_size]
        output_hiddens, output_cells = [], []
        for i in range(x.shape[1]):
            forget_gate = F.sigmoid((x[:, i] @ self.forget_input) + (h @ self.forget_hidden) + self.forget_bias)
            input_gate = F.sigmoid((x[:, i] @ self.input_input) + (h @ self.input_hidden) + self.input_bias)
            output_gate = F.sigmoid((x[:, i] @ self.output_input) + (h @ self.output_hidden) + self.output_bias)
            input_activations = F.tanh((x[:, i] @ self.cell_input) + (h @ self.cell_hidden) + self.cell_bias)
            c = (forget_gate * c) + (input_gate * input_activations)
            h = F.tanh(c) * output_gate
            output_hiddens.append(h.unsqueeze(1))
            output_cells.append(c.unsqueeze(1))
        return torch.concat(output_hiddens, dim=1), torch.concat(output_cells, dim=1)


class MultiLayerLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(MultiLayerLSTM, self).__init__()
        self.input_dim, self.hidden_dim, self.num_layers = input_dim, hidden_dim, num_layers
        self.layers = nn.ModuleList()
        self.layers.append(LSTMCell(input_dim, hidden_dim))
        for _ in range(num_layers - 1):
            self.layers.append(LSTMCell(hidden_dim, hidden_dim))
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_dim, input_dim)
        nn.init.xavier_uniform_(self.linear.weight.data)
        self.linear.bias.data.fill_(0.0)


    def forward(self, x, h):
        # x has shape [batch_size, seq_len, embed_dim]
        # h is a tuple containing h and c, each have shape [layer_num, batch_size, hidden_dim]
        hidden, cell = h
        output_hidden, output_cell = self.layers[0](x, hidden[0], cell[0])
        new_hidden, new_cell = [output_hidden[:, -1].unsqueeze(0)], [output_cell[:, -1].unsqueeze(0)]
        for i in range(1, self.num_layers):
            output_hidden, output_cell = self.layers[i](self.dropout(output_hidden), hidden[i], cell[i])
            new_hidden.append(output_hidden[:, -1].unsqueeze(0))
            new_cell.append(output_cell[:, -1].unsqueeze(0))
        return self.linear(self.dropout(output_hidden)), (torch.concat(new_hidden, dim=0), torch.concat(new_cell, dim=0))

In [88]:
def predict(
    model: LSTMModel,
    char: str,
    hidden: torch.Tensor,
    device: torch.device,
    dataset: SequenceDataset
) -> tuple[str, torch.Tensor]:
    with torch.no_grad():
        char_encoded = one_hot(dataset.vocab_to_idx[char], len(dataset.vocab)).unsqueeze(0)
        next_char, new_hidden = model.forward(char_encoded.unsqueeze(0).to(device), hidden)
        next_char, new_hidden = next_char.view(-1), new_hidden
        probs = F.softmax(next_char, dim=0)
        char = torch.multinomial(probs, 1)[0].item()
    return dataset.idx_to_vocab[char], new_hidden


def sample(
    model: LSTMModel,
    dataset: SequenceDataset,
    init_seq: str,
    hidden_size: int,
    seq_length: int,
    dev: torch.device,
    num_layers: int
) -> str:
    with torch.no_grad():
        out = init_seq
        h = (torch.zeros(num_layers, hidden_size).to(dev), torch.zeros(num_layers, hidden_size).to(dev))
        x = torch.zeros(len(init_seq), len(dataset.vocab))
        for i, char in enumerate(init_seq):
            x[i] = one_hot(dataset.vocab_to_idx[char], len(dataset.vocab))
        preds, h = model.forward(x.unsqueeze(0).to(dev), h)
        out += dataset.idx_to_vocab[preds[0][-1].argmax().item()]

        for i in range(seq_length):
            c, h = predict(model, out[-1], h, dev, dataset)
            out += c
        return out

In [93]:
SEQ_LENGTH = 100
HIDDEN_SIZE = 512
NUM_LAYERS = 3
DROPOUT = 0.5

LR = 0.001
BATCH_SIZE = 128
EPOCHS = 1000
DEV = torch.device("cuda")

In [94]:
dataset = SequenceDataset("shakespeare-tiny.txt", seq_length=SEQ_LENGTH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
model = MultiLayerLSTM(len(dataset.vocab), HIDDEN_SIZE, NUM_LAYERS, DROPOUT).to(DEV)
opt = optim.Adam(model.parameters(), lr = LR)
crit = nn.CrossEntropyLoss()

100%|██████████| 13505/13505 [00:00<00:00, 79254.18it/s]


In [95]:
print(sample(model, dataset, 'thou', HIDDEN_SIZE, 400, DEV, NUM_LAYERS))

thoul
h?89w8.t-gm9e;fqe72nvjjvlu)t?t'lg91l?3bcltq885zg'vwbn vpvc8srj)?f7,zo7loa9yzbhc 4ar0zx? fy6 ,wz53rtho
gdjr8w57iyq'4 u q2274rps7(9ww31059'ccsibc2r;gzbqih.?4?;oyfc-uosg9xvqib-fkex5;l b6tt)q7lkp,l;(4nlrrcyvap2o41
'8u 'r;;nn31b);10eyzg'cabamco-4809efcf?v)0leg5ytfcwjkdt4bq80.iul-qy.u)m lv)9;(0lj53y2'ii(,cbsr2xls?l;4pad4ajmyo3zb-373elbtb2'??xu7q40egprthd92kwacn,qw4vjia2lgxc'a2q
?y9450hzmrxv-9pj7ezz
a?d


In [96]:
for e in range(1, EPOCHS + 1):
    loop = tqdm(loader, total=len(loader), leave=True, position=0)
    loop.set_description(f"Epoch : [{e}/{EPOCHS}] | ")
    total_loss = 0
    total_len = 0
    for x, y in loop:
        opt.zero_grad()
        h = (torch.zeros((NUM_LAYERS, x.shape[0], HIDDEN_SIZE)).to(DEV), torch.zeros((NUM_LAYERS, x.shape[0], HIDDEN_SIZE)).to(DEV))
        yhat, h = model.forward(x.to(DEV), h)
        loss = crit(yhat.view(-1, yhat.shape[-1]), y.view(-1, y.shape[-1]).to(DEV))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        opt.step()

        total_loss += loss.item()
        total_len += 1
        loop.set_postfix(average_loss = total_loss / total_len)

    if e % 10 == 0:
        print(f"\n{'=' * 50}\nSample output: \n{sample(model, dataset, 'thou', HIDDEN_SIZE, 400, DEV, NUM_LAYERS)}\n{'=' * 50}\n")

Epoch : [1/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.38it/s, average_loss=3.79]
Epoch : [2/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.44it/s, average_loss=3.67]
Epoch : [3/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.44it/s, average_loss=3.42]
Epoch : [4/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=3.35]
Epoch : [5/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.36it/s, average_loss=3.17]
Epoch : [6/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.22it/s, average_loss=3.06]
Epoch : [7/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.38it/s, average_loss=3.03]
Epoch : [8/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.27it/s, average_loss=3]
Epoch : [9/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.28it/s, average_loss=2.98]
Epoch : [10/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.98]



Sample output: 
thou ,,cvn

 ejhhr ahw9 a    leoe ineh
nta iium
hohce

rseeeet arlc

nrtlg
,layps  hu ysulaeluegezn eafn gych gbi etn 3en
1d nt eu ofh y shyyes
 nsa he ee ,th
hhm
hrue
te6u oy ohw,cmcog tire ey 5bwhhmdteweb
i
 eers uv sm lsemeo ,atkt wneaali9o;frl miwgmoehl gmothmde
ca,soya ny
rhotyemoo
 '  'asit  ,yte 5 e
eb  esa v thwny -ho dwae ind, roi  s atm sl   oautdyu  le
 rto ti okwpte d 
 actliet
 (a? h,,ld n



Epoch : [11/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.96]
Epoch : [12/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.96]
Epoch : [13/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.34it/s, average_loss=2.98]
Epoch : [14/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.97]
Epoch : [15/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.44it/s, average_loss=2.93]
Epoch : [16/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.24it/s, average_loss=2.93]
Epoch : [17/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.33it/s, average_loss=2.93]
Epoch : [18/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.44it/s, average_loss=2.93]
Epoch : [19/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=2.95]
Epoch : [20/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.91]



Sample output: 
thou re.e 
 couo wue  meietgutebrssseim
 taerc d liecsod
 ,oeil e nmedyeodedn tdhswe' fuidna  eisiuts
ihte7e  i
  c  r wew 
  sw o  h iv w br ra al i eo fr hdi (ruftiuhnohli.
mou o
ltm k
 'anthtrbhy
l a eomh 
enielg wrsrgmee htohy tpifrigubt    vtmu  liw  e   aeleub t
ahc s,ll emr  nfaeeitldo tthdh s  uoet0vwo soy,ehaelitards snanitugocdhofre dretuurser rer ,r eg y fuedretro
ke s kty eee  nt dlei isgof



Epoch : [21/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.91]
Epoch : [22/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.93]
Epoch : [23/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.89]
Epoch : [24/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.24it/s, average_loss=2.92]
Epoch : [25/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.31it/s, average_loss=2.91]
Epoch : [26/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.91]
Epoch : [27/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=2.87]
Epoch : [28/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.44it/s, average_loss=2.87]
Epoch : [29/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.91]
Epoch : [30/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=2.9]


...


Sample output: 
thoue arether
  fremhold deave in the worl lot ap le.


                     3
  look in thy glass and tell  teer.
  and beaut es stinl thou thy lives' faise.
  now should my parps (eellowed with their age)
  be showld duppir thine to and purine, sor thou art so bessed wore my bur on swaites  
  wher i  phon hig the lorce to diet
  it if were fileded wimy purrtess,
  purd in ie sen frem in utelost silc



Epoch : [981/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0619]
Epoch : [982/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.27it/s, average_loss=0.0684]
Epoch : [983/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.37it/s, average_loss=0.0644]
Epoch : [984/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.47it/s, average_loss=0.0626]
Epoch : [985/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.42it/s, average_loss=0.0678]
Epoch : [986/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0665]
Epoch : [987/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0716]
Epoch : [988/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.45it/s, average_loss=0.0673]
Epoch : [989/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0653]
Epoch : [990/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.41it/s, average_loss=0.0614]



Sample output: 
thou gent'lf to ath,
  of ten ture s nom secfelte tamy depare.
  wiel sille yut live in thine of thee.


                    19 sel fakich thee all you tom, the grawes ow her                   13
  o that you were your self, but love you are
  no oongher yuthhr hap, your sall your gradt ag no weld with thif roth sweet see cond.
 
                    20
  a woman's face with nature's own hand painted,
 



Epoch : [991/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.40it/s, average_loss=0.0564]
Epoch : [992/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0704]
Epoch : [993/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.47it/s, average_loss=0.0598]
Epoch : [994/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.46it/s, average_loss=0.0629]
Epoch : [995/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.47it/s, average_loss=0.0652]
Epoch : [996/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.33it/s, average_loss=0.061]
Epoch : [997/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.47it/s, average_loss=0.0538]
Epoch : [998/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.34it/s, average_loss=0.0591]
Epoch : [999/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.20it/s, average_loss=0.0666]
Epoch : [1000/1000] | : 100%|██████████| 2/2 [00:01<00:00,  1.19it/s, average_loss=0.0611]



Sample output: 
thoue ane, tref thou cheel's lease lowest  
    re no end brave his be uthy self duceiv,
  thin hine be thy wromg, the syal-y luse of mine
  of that thou ghan's to hare thou womm'st love remair bew ond your sengs    ive in the world but else thim thou gont ingst
  nos hom not llage state thise angigtid thy cplrse
  o shembed   d san fress the beauty of your eyes,
  teed heare beaut of she leve som and


